In [1]:
import os

In [2]:
%pwd

'c:\\Users\\MAINGATE\\Desktop\\Maize_disease_classification\\end-to-end_mazie_disease_classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\MAINGATE\\Desktop\\Maize_disease_classification\\end-to-end_mazie_disease_classification'

In [5]:
import tf_keras as tf

In [6]:
model= tf.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: tuple
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

import ast


In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):

        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    



    def get_validation_config(self) -> EvaluationConfig:

        eval_config= EvaluationConfig(
            path_of_model= "artifacts/training/model.h5",
            training_data= "artifacts/data_ingestion/Maize_leaf_dataset",
            all_params= self.params,
            params_image_size= ast.literal_eval(self.params.IMAGE_SIZE),
            params_batch_size= self.params.BATCH_SIZE
        )

        return eval_config

In [10]:
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config=config


    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split= 0.30
        )

        dataflow_kwargs= dict(
            target_size= self.config.params_image_size[:2],
            batch_size= self.config.params_batch_size,
            interpolation= "bilinear"
        )

        valid_datagenerator= tf.preprocessing.image.ImageDataGenerator(
             **datagenerator_kwargs
        )

        self.valid_generator= valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset= "validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.Model:
        return tf.models.load_model(path)

    def evaluation(self):

        self.model= self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score= model.evaluate(self.valid_generator)


    def save_score(self):
        scores= {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path= Path("scores.json"), data=scores)



In [12]:
try: 
    config= ConfigurationManager()
    val_config= config.get_validation_config()
    evaluation= Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e

[2025-05-07 12:16:46,454: INFO: common: yaml file: (path_to_yaml) loaded successfully]
[2025-05-07 12:16:46,773: INFO: common: yaml file: (path_to_yaml) loaded successfully]
[2025-05-07 12:16:46,780: INFO: common: created a directory at : artifacts]
Found 2805 images belonging to 3 classes.

[2025-05-07 12:19:00,820: WARNING: module_wrapper: From c:\Users\MAINGATE\.conda\envs\maize\lib\site-packages\tensorflow\python\autograph\converters\directives.py:126: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
88/88 [==============================] - 929s 9s/step - loss: 0.2737 - accuracy: 0.9169
[2025-05-07 12:32:41,428: INFO: common: json file saved at: scores.json]
